<a href="https://colab.research.google.com/github/LeonardoClemente23/predicao-titanic/blob/main/sklearn_titanic_kaggle_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-optimize
!pip install category_encoders

     |████████████████████████████████| 81kB 2.2MB/s 
     |████████████████████████████████| 81kB 2.2MB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Modelos testados

In [ ]:
#modelo 1
from sklearn.ensemble import RandomForestClassifier
#modelo 2
from sklearn.ensemble import GradientBoostingClassifier
#modelo 3
from sklearn.ensemble import ExtraTreesClassifier
#modelo 4
from sklearn.linear_model import LogisticRegressionCV
#modelo 5
from sklearn.neighbors import KNeighborsClassifier
#modelo 6 e 7
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
# modelo 8
from sklearn.linear_model import LogisticRegression 
#modelo 9
from lightgbm import LGBMClassifier
#modelo 10
from xgboost import XGBClassifier


In [ ]:
# Teste de eficiencia do modelo
from sklearn.metrics import classification_report, confusion_matrix
# Score do torneio
from sklearn.metrics import accuracy_score
# Dividindo teste e validação
from sklearn.model_selection import train_test_split

## Abrindo o dataset

In [ ]:
path = 'drive/My Drive/Colab Notebooks/Datasets/titanic/'

train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')


In [ ]:
#train = train.drop(339)

In [ ]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
train.head(7)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S


## Análise e tratamento do dataset

In [ ]:
# verificando se tem valores faltantes
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
# mudando os valores nan por -1
train = train.fillna(-1)
test = test.fillna(-1)
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [ ]:
# verificando os tipos dos dados
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  890 non-null    int64  
 1   Survived     890 non-null    int64  
 2   Pclass       890 non-null    int64  
 3   Name         890 non-null    object 
 4   Sex          890 non-null    object 
 5   Age          890 non-null    float64
 6   SibSp        890 non-null    int64  
 7   Parch        890 non-null    int64  
 8   Ticket       890 non-null    object 
 9   Fare         890 non-null    float64
 10  Cabin        890 non-null    object 
 11  Embarked     890 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 90.4+ KB


In [ ]:
# mudando para tipo categórico 'Embarked' e 'Sex'
train = train.astype({'Embarked': 'category','Sex':'category','Cabin':'str'})
test = test.astype({'Embarked': 'category','Sex':'category','Cabin':'str'})


In [ ]:
# Função usada para extrair o deck de onde o passageiro estava a partir da coluna cabin
def letter_number_split(string):
  if string.isalnum():
    return string[0]
  else:
    return 0

In [ ]:
# Extraindo a feature do deck do navio
train['Deck'] = train['Cabin'].map(lambda x: letter_number_split(x))
train = train.astype({'Deck':'category'})

test['Deck'] = test['Cabin'].map(lambda x: letter_number_split(x))
test = test.astype({'Deck':'category'})

In [ ]:
# Extraindo os títulos dos passageiros
train['Name_title'] = train['Name'].str.split(", ", expand = True)[1].str.split(". ", expand = True)[0]
test['Name_title'] = test['Name'].str.split(", ", expand = True)[1].str.split(". ", expand = True)[0]

print(train['Name_title'].unique())
print("-"*70)
print(test['Name_title'].unique())

['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'th' 'Jonkheer']
----------------------------------------------------------------------
['Mr' 'Mrs' 'Miss' 'Master' 'Ms' 'Col' 'Rev' 'Dr' 'Dona']


In [ ]:
# Substituindo os titulos mais incomuns por 'Misc'
#min_titles = (train["Name_title"].value_counts() < 10)
#train['Name_title'] = train['Name_title'].apply(lambda x: 'Misc' if min_titles.loc[x] else x)

#min_titles = (test["Name_title"].value_counts() < 10)
#test['Name_title'] = test['Name_title'].apply(lambda x: 'Misc' if min_titles.loc[x] else x)

In [ ]:
# Categorizando os títulos
train = train.astype({'Name_title':'category'})

test = test.astype({'Name_title':'category'})

In [ ]:
# Adicionando a feature de tamanho da família preço da passagem por pessoa
train['Family_size'] = train['SibSp'] + train['Parch']
train['Fare_per_person'] = train['Fare'] / (train['Family_size'] + 1)

test['Family_size'] = test['SibSp'] + test['Parch']
test['Fare_per_person'] = test['Fare'] / (test['Family_size'] + 1)

In [ ]:
#Categorizando idade
train['Age_band'] = pd.cut(train['Age'],bins=[-1, 0, 15, 31, 47,63,80])
test['Age_band'] = pd.cut(test['Age'],bins=[-1, 0, 15, 31, 47,63,80])

In [ ]:
# Categorizando tarifa por pessoa
train['Fare_per_person_band'] = pd.qcut(train['Fare_per_person'],6)
test['Fare_per_person_band'] = pd.qcut(test['Fare_per_person'],6)

In [ ]:
# categorizando Tarifa
train['Fare_band'] = pd.qcut(train['Fare'],6)
test['Fare_band'] = pd.qcut(test['Fare'],6)

In [ ]:
train = train.astype({'Fare_band':'category'})
test = test.astype({'Fare_band':'category'})

In [ ]:
# A pessoa estava sozinha ou não, no navio? 1 = sim. 0 = não
train['Is_alone'] = [1 if train['Family_size'][ind] == 0 else 0 for ind in train.index]
test['Is_alone'] = [1 if test['Family_size'][ind] == 0 else 0 for ind in test.index]

In [ ]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck,Name_title,Family_size,Fare_per_person,Age_band,Fare_per_person_band,Fare_band,Is_alone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,-1,S,0,Mr,1,3.62500,"(15, 31]","(-0.001, 6.75]","(-0.001, 7.775]",0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C,Mrs,1,35.64165,"(31, 47]","(29.582, 512.329]","(52.462, 512.329]",0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,-1,S,0,Miss,0,7.92500,"(15, 31]","(7.775, 8.229]","(7.775, 8.662]",1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,C,Mrs,1,26.55000,"(31, 47]","(13.0, 29.582]","(52.462, 512.329]",0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,-1,S,0,Mr,0,8.05000,"(31, 47]","(7.775, 8.229]","(7.775, 8.662]",1


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890 entries, 0 to 890
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   PassengerId           890 non-null    int64   
 1   Survived              890 non-null    int64   
 2   Pclass                890 non-null    int64   
 3   Name                  890 non-null    object  
 4   Sex                   890 non-null    category
 5   Age                   890 non-null    float64 
 6   SibSp                 890 non-null    int64   
 7   Parch                 890 non-null    int64   
 8   Ticket                890 non-null    object  
 9   Fare                  890 non-null    float64 
 10  Cabin                 890 non-null    object  
 11  Embarked              890 non-null    category
 12  Deck                  890 non-null    category
 13  Name_title            890 non-null    category
 14  Family_size           890 non-null    int64   
 15  Fare_p

In [ ]:
# Variáveis categóricas do dataset pós tratamento
train.columns[train.dtypes == 'category'].to_list()

['Sex',
 'Embarked',
 'Deck',
 'Name_title',
 'Age_band',
 'Fare_per_person_band',
 'Fare_band']

In [ ]:
# Encode de variáveis 
train['Age_band_code'] = train['Age_band'].cat.codes
train['Fare_per_person_band_code'] = train['Fare_per_person_band'].cat.codes
train['Fare_band_code'] = train['Fare_band'].cat.codes

test['Age_band_code'] = test['Age_band'].cat.codes
test['Fare_per_person_band_code'] = test['Fare_per_person_band'].cat.codes
test['Fare_band_code'] = test['Fare_band'].cat.codes

In [ ]:
# Variável idade_encode x classe
train['Age_band_code*Class'] = train['Pclass'] * train['Age_band_code']
test['Age_band_code*Class'] = test['Pclass'] * test['Age_band_code']

In [ ]:
# Treino com variáveis categóricas no  formato one hot encode
from category_encoders.one_hot import OneHotEncoder

encoder = OneHotEncoder(cols=['Sex','Embarked','Deck','Name_title','Age_band_code','Fare_per_person_band_code','Fare_band_code'], use_cat_names=True)
train_ohe = encoder.fit_transform(train)
encoder.fit_transform(train.drop(columns='Survived'), train['Survived'])
test_ohe = encoder.transform(test)
test_ohe.head()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,PassengerId,Pclass,Name,Sex_male,Sex_female,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_S,Embarked_C,Embarked_Q,Embarked_-1,Deck_0,Deck_C,Deck_E,Deck_G,Deck_D,Deck_A,Deck_B,Deck_F,Name_title_Mr,Name_title_Mrs,Name_title_Miss,Name_title_Master,Name_title_Don,Name_title_Rev,Name_title_Dr,Name_title_Mme,Name_title_Ms,Name_title_Major,Name_title_Lady,Name_title_Sir,Name_title_Mlle,Name_title_Col,Name_title_Capt,Name_title_th,Name_title_Jonkheer,Family_size,Fare_per_person,Age_band,Fare_per_person_band,Fare_band,Is_alone,Age_band_code_2.0,Age_band_code_3.0,Age_band_code_-1.0,Age_band_code_4.0,Age_band_code_1.0,Age_band_code_5.0,Fare_per_person_band_code_0.0,Fare_per_person_band_code_5.0,Fare_per_person_band_code_2.0,Fare_per_person_band_code_4.0,Fare_per_person_band_code_3.0,Fare_per_person_band_code_1.0,Fare_band_code_0.0,Fare_band_code_5.0,Fare_band_code_1.0,Fare_band_code_4.0,Fare_band_code_3.0,Fare_band_code_2.0,Age_band_code*Class
0,892,3,"Kelly, Mr. James",1,0,34.5,0,0,330911,7.8292,-1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.829200,"(31, 47]","(7.775, 8.662]","(7.75, 8.662]",1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,9
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,1,47.0,1,0,363272,7.0000,-1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.500000,"(31, 47]","(-1.001, 7.225]","(-1.001, 7.75]",0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,9
2,894,2,"Myles, Mr. Thomas Francis",1,0,62.0,0,0,240276,9.6875,-1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9.687500,"(47, 63]","(8.662, 13.0]","(8.662, 14.454]",1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,8
3,895,3,"Wirz, Mr. Albert",1,0,27.0,0,0,315154,8.6625,-1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.662500,"(15, 31]","(7.775, 8.662]","(7.75, 8.662]",1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,6
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,1,22.0,1,1,3101298,12.2875,-1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,4.095833,"(15, 31]","(-1.001, 7.225]","(8.662, 14.454]",0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,6


In [ ]:
# Treino com variáveis categóricas no  formato target encode
from category_encoders.target_encoder import TargetEncoder

encoder = TargetEncoder(cols=['Sex','Embarked','Deck','Name_title','Age_band_code','Fare_per_person_band_code','Fare_band_code'])
train_tg = encoder.fit_transform(train, train['Survived'])
encoder.fit_transform(train.drop(columns='Survived'), train['Survived'])
test_tg = encoder.transform(test)
train_tg.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck,Name_title,Family_size,Fare_per_person,Age_band,Fare_per_person_band,Fare_band,Is_alone,Age_band_code,Fare_per_person_band_code,Fare_band_code,Age_band_code*Class
0,1,0,3,"Braund, Mr. Owen Harris",0.189236,22.0,1,0,A/5 21171,7.2500,-1,0.337481,0.309423,0.156977,1,3.62500,"(15, 31]","(-0.001, 6.75]","(-0.001, 7.775]",0,0.362319,0.291391,0.205128,6
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0.742038,38.0,1,0,PC 17599,71.2833,C85,0.553571,0.607843,0.792000,1,35.64165,"(31, 47]","(29.582, 512.329]","(52.462, 512.329]",0,0.403061,0.651007,0.697987,3
2,3,1,3,"Heikkinen, Miss. Laina",0.742038,26.0,0,0,STON/O2. 3101282,7.9250,-1,0.337481,0.309423,0.697802,0,7.92500,"(15, 31]","(7.775, 8.229]","(7.775, 8.662]",1,0.362319,0.195652,0.190789,6
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0.742038,35.0,1,0,113803,53.1000,C123,0.337481,0.607843,0.792000,1,26.55000,"(31, 47]","(13.0, 29.582]","(52.462, 512.329]",0,0.403061,0.491228,0.697987,3
4,5,0,3,"Allen, Mr. William Henry",0.189236,35.0,0,0,373450,8.0500,-1,0.337481,0.309423,0.156977,0,8.05000,"(31, 47]","(7.775, 8.229]","(7.775, 8.662]",1,0.403061,0.195652,0.190789,9


In [ ]:
# # Treino com variáveis categóricas no formato ordinal encode
train_ord = train.copy()
test_ord = test.copy()
train_ord['Sex_code'] = train['Sex'].cat.codes
train_ord['Embarked_code'] = train['Embarked'].cat.codes
train_ord['Deck_codes'] = train['Deck'].cat.codes
train_ord['Name_title_code'] = train['Name_title'].cat.codes
train_ord['Age_band_code'] = train['Age_band'].cat.codes
train_ord['Fare_per_person_band_code'] = train['Fare_per_person_band'].cat.codes
train_ord['Fare_band_code'] = train['Fare_band'].cat.codes

test_ord['Sex_code'] = test['Sex'].cat.codes
test_ord['Embarked_code'] = test['Embarked'].cat.codes
test_ord['Deck_codes'] = test['Deck'].cat.codes
test_ord['Name_title_code'] = test['Name_title'].cat.codes
test_ord['Age_band_code'] = test['Age_band'].cat.codes
test_ord['Fare_per_person_band_code'] = test['Fare_per_person_band'].cat.codes
test_ord['Fare_band_code'] = test['Fare_band'].cat.codes

train_ord.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck,Name_title,Family_size,Fare_per_person,Age_band,Fare_per_person_band,Fare_band,Is_alone,Age_band_code,Fare_per_person_band_code,Fare_band_code,Age_band_code*Class,Sex_code,Embarked_code,Deck_codes,Name_title_code
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,-1,S,0,Mr,1,3.62500,"(15, 31]","(-0.001, 6.75]","(-0.001, 7.775]",0,2,0,0,6,1,3,0,11
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C,Mrs,1,35.64165,"(31, 47]","(29.582, 512.329]","(52.462, 512.329]",0,3,5,5,3,0,1,3,12
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,-1,S,0,Miss,0,7.92500,"(15, 31]","(7.775, 8.229]","(7.775, 8.662]",1,2,2,1,6,0,3,0,8
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,C,Mrs,1,26.55000,"(31, 47]","(13.0, 29.582]","(52.462, 512.329]",0,3,4,5,3,0,3,3,12
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,-1,S,0,Mr,0,8.05000,"(31, 47]","(7.775, 8.229]","(7.775, 8.662]",1,3,2,1,9,1,3,0,11


## Setando os datasets de treino

In [ ]:
# Raw dataset
Xtrain = train.select_dtypes(include='number').drop(['Survived','PassengerId'], axis=1)
ytrain = train['Survived']

# one hot encode dataset
Xtrain_ohe = train_ohe.select_dtypes(include='number').drop(['Survived','PassengerId'], axis=1)
ytrain_ohe = train_ohe['Survived']

# ordinal encode dataset
Xtrain_ord = train_ord.select_dtypes(include='number').drop(['Survived','PassengerId'], axis=1)
ytrain_ord = train_ord['Survived']

# target encode dataset
Xtrain_tg = train_tg.select_dtypes(include='number').drop(['Survived','PassengerId'], axis=1)
ytrain_tg = train_tg['Survived']

Xtest = test.select_dtypes(include='number').drop('PassengerId', axis=1)

In [ ]:
# dataset de treino com todos os encoders juntos
train_gg = train.copy()
train_gg = pd.merge(train_gg,train_ord, on='PassengerId',suffixes=(' ',' '),)
train_gg = pd.merge(train_gg,train_ohe, on='PassengerId',suffixes=(' ',' '))
train_gg = pd.merge(train_gg,train_tg, on='PassengerId',suffixes=(' ',' '))

test_gg = test.copy()
test_gg = pd.merge(test_gg,test_ord, on='PassengerId',suffixes=(' ',' '))
test_gg = pd.merge(test_gg,test_ohe, on='PassengerId',suffixes=(' ',' '))
test_gg = pd.merge(test_gg,test_tg, on='PassengerId',suffixes=(' ',' '))


In [ ]:
# Verificando existência de colunas duplas
train_gg = train_gg.loc[:,~train_gg.columns.duplicated()]
test_gg = test_gg.loc[:,~test_gg.columns.duplicated()]

In [ ]:
Xtrain_gg = train_gg.select_dtypes(include='number').drop(columns=['PassengerId','Survived '])
ytrain_gg = train_gg['Survived ']

Xtest_gg = test_gg.select_dtypes(include='number').drop('PassengerId', axis=1)

In [ ]:
# Split treino/validação
X_train, X_val, y_train, y_val = train_test_split(Xtrain_gg, ytrain_gg, test_size=0.25, random_state=0)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((667, 72), (223, 72), (667,), (223,))

## Início dos treinos

### Teste primário dos modelos

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(Xtrain_gg, ytrain_gg, test_size=0.3, random_state=0)

dict_models = {'Random Forest Classifier':RandomForestClassifier(), 
               'Gradient Boosting Classifier':GradientBoostingClassifier(),
               'Extra Trees Classifier':ExtraTreesClassifier(),
               'K Neighbors Classifier':KNeighborsClassifier(),
               'Decision Tree Classifier':DecisionTreeClassifier(),
               'Extra Tree Classifier':ExtraTreeClassifier(), 
               'LGBM Classifier':LGBMClassifier(),
               'XGB Classifier':XGBClassifier()}

for key, modelo in dict_models.items():
  modelo.fit(X_train, y_train)
  
  pred = modelo.predict(X_val)

  score = accuracy_score(y_val, pred)

  print(f'Modelo: {key} - score: {score}')

### Train normal:
- Modelo: Random Forest Classifier - score: 0.6764044943820224
- Modelo: Gradient Boosting Classifier - score: 0.701123595505618
- Modelo: Extra Trees Classifier - score: 0.6741573033707865
- Modelo: K Neighbors Classifier - score: 0.6629213483146067
- Modelo: Decision Tree Classifier - score: 0.6494382022471911
- Modelo: Extra Tree Classifier - score: 0.6539325842696629
- Modelo: LGBM Classifier - score: 0.6808988764044944
- Modelo: XGB Classifier - score: 0.6966292134831461

### Train One Hot Encode
- Modelo: Random Forest Classifier - score: 0.7775280898876404
- Modelo: Gradient Boosting Classifier - score: 0.7955056179775281
- Modelo: Extra Trees Classifier - score: 0.7528089887640449
- Modelo: K Neighbors Classifier - score: 0.6966292134831461
- Modelo: Decision Tree Classifier - score: 0.7438202247191011
- Modelo: Extra Tree Classifier - score: 0.7370786516853932
- Modelo: LGBM Classifier - score: 0.7820224719101123
- Modelo: XGB Classifier - score: 0.802247191011236

### Train Ordinal
- Modelo: Random Forest Classifier - score: 0.7707865168539326
- Modelo: Gradient Boosting Classifier - score: 0.7955056179775281
- Modelo: Extra Trees Classifier - score: 0.750561797752809
- Modelo: K Neighbors Classifier - score: 0.6921348314606741
- Modelo: Decision Tree Classifier - score: 0.7393258426966293
- Modelo: Extra Tree Classifier - score: 0.7460674157303371
- Modelo: LGBM Classifier - score: 0.7797752808988764
- Modelo: XGB Classifier - score: 0.8

### Train Target Encode 
- Modelo: Random Forest Classifier - score: 0.7775280898876404
- Modelo: Gradient Boosting Classifier - score: 0.797752808988764
- Modelo: Extra Trees Classifier - score: 0.7460674157303371
- Modelo: K Neighbors Classifier - score: 0.6651685393258427
- Modelo: Decision Tree Classifier - score: 0.7438202247191011
- Modelo: Extra Tree Classifier - score: 0.7528089887640449
- Modelo: LGBM Classifier - score: 0.7932584269662921
- Modelo: XGB Classifier - score: 0.7932584269662921

### Train geral junto
- Modelo: Random Forest Classifier - score: 0.7640449438202247
- Modelo: Gradient Boosting Classifier - score: 0.797752808988764
- Modelo: Extra Trees Classifier - score: 0.750561797752809
- Modelo: K Neighbors Classifier - score: 0.6966292134831461
- Modelo: Decision Tree Classifier - score: 0.7438202247191011
- Modelo: Extra Tree Classifier - score: 0.7393258426966293
- Modelo: LGBM Classifier - score: 0.7820224719101123
- Modelo: XGB Classifier - score: 0.797752808988764

### Hiperparametrização dos modelos com melhores resultados preliminares 

In [ ]:
# Otimização do Modelo 1
# scores baseline: 0.7775280898876404

from skopt import BayesSearchCV

hp_params_dict1 = dict(max_depth=(4,20), min_samples_split=(0.05,1.0),
                          min_samples_leaf=(0.05,0.5), max_leaf_nodes=(2,128),
                          min_impurity_decrease=(0.0,1.0), min_weight_fraction_leaf=(0.0,0.5))

for class_weight in ['balanced', 'balanced_subsample']:
  for criterion in ['gini', 'entropy']:
    opt1 = BayesSearchCV(RandomForestClassifier(n_estimators=500,random_state=0,
                                                class_weight=class_weight,
                                                criterion=criterion),
                         hp_params_dict1, n_jobs=-1, n_iter=50, cv=2, verbose=0)

    opt1.fit(Xtrain_gg,ytrain_gg) 

    print(f'class_weight -> {class_weight}, criterion -> {criterion}, best score = {opt1.best_score_},\nmelhores params = {opt1.best_params_}')


In [ ]:
# Otimização do Modelo 2
# score baseline: 0.797752808988764

# loss{‘deviance’, ‘exponential’}, default=’deviance’
# criterion{‘friedman_mse’, ‘mse’, ‘mae’}, default=’friedman_mse’

hp_params_dict2 = dict(learning_rate = (0.001,0.01,'log-uniform'), 
                          min_samples_split = (0.25,0.7), 
                          min_samples_leaf = (0.05,0.35), max_depth = (4,20))

for loss in ['deviance', 'exponential']:
  for criterion in ['friedman_mse', 'mse', 'mae']:
    opt2 = BayesSearchCV(GradientBoostingClassifier(n_estimators=500,
                                                    random_state=0,loss=loss,
                                                    criterion=criterion),
                         hp_params_dict2, n_jobs=-1, n_iter=50, cv=2, verbose=0)

    opt2.fit(Xtrain_gg,ytrain_gg)

    print(f'loss -> {loss}, criterion -> {criterion}, best score = {opt2.best_score_},\nmelhores params = {opt2.best_params_}')

In [ ]:
# Otimização do Modelo 3
# score baseline: 0.7528089887640449

hp_params_dict3 = dict(max_depth=(4, 20), min_samples_split=(0.01, 1.0),
                       min_samples_leaf=(0.01, 0.5),
                       min_weight_fraction_leaf=(0.0, 0.5))

for criterion in ['gini', 'entropy']:
  for class_weight in ['balanced', 'balanced_subsample']:
    opt3 = BayesSearchCV(ExtraTreesClassifier(n_estimators=300, random_state=0),
                         hp_params_dict3, n_jobs=-1, cv=2, n_iter=50)
    
    opt3.fit(Xtrain_gg,ytrain_gg)

    print(f'class_weight -> {class_weight}, criterion -> {criterion}, best score = {opt1.best_score_},\nmelhores params = {opt1.best_params_}')

In [ ]:
# Otimização do Modelo 9
# score baseline: 0.7932584269662921

hp_params_dict9 = dict(learning_rate=(1e-3,1e-1),num_leaves=(2,128),
                      min_child_samples=(1,100), subsample=(0.05,1.0),
                      colsample_bytree=(0.1,1.0))

opt9 = BayesSearchCV(LGBMClassifier(n_estimators=300, random_state=0),hp_params_dict9,
                    n_jobs=-1, n_iter=200, cv=2, verbose=0)

opt9.fit(Xtrain_gg,ytrain_gg)

print(f'best score = {opt9.best_score_}, melhores params = {opt9.best_params_}')


In [ ]:
# Otimização do Modelo 10
# score baseline: 0.802247191011236

hp_params_dict10 = dict(max_depth=(4,20), subsample=(0.1,1.0),
                        colsample_bylevel=(0.1,1.0), colsample_bytree=(0.1,1.0),
                        min_child_weight=(1,30), reg_alpha=(0.0,1.0), reg_lambda=(0.0,1.0),
                        learning_rate=(0.001, 0.1))

opt10 = BayesSearchCV(XGBClassifier(n_estimators=300, random_state=0),
                      hp_params_dict10, n_jobs=-1, n_iter=200, cv=2, verbose=0)

opt10.fit(Xtrain_gg,ytrain_gg)

print(f'best score = {opt10.best_score_}, melhores params = {opt10.best_params_}')


# **Melhores resultados**

- ## Random Forest Classifier

  - **class_weight** -> balanced,
    
    **criterion** -> entropy,
    
    **best score**: 0.7887640449438202,
    
    **melhores params**:
    - (**max_depth**, 5),
    - (**max_leaf_nodes**, 45),
    - (**min_impurity_decrease**, 0.09134778204836073),
    - (**min_samples_leaf**, 0.08845872709942093),
    - (**min_samples_split**, 0.07163235347452308), 
    - (**min_weight_fraction_leaf**, 0.34402341231767675)

  - **class_weight** -> balanced_subsample,
    
    **criterion** -> gini,
    
    **best score** = 0.7887640449438202,
    
    **melhores params**:
    - (**max_depth**, 19),
    - (**max_leaf_nodes**, 89),
    - (**min_impurity_decrease**, 0.09469958749590013),
    - (**min_samples_leaf**, 0.05),
    - (**min_samples_split**, 0.08980401552521708),
    - (**min_weight_fraction_leaf**, 0.32355998101957956)

  - **class_weight** -> balanced_subsample,
    
    **criterion** -> entropy,
    
    **best score** = 0.7887640449438202,
    
    **melhores params**:
    - (**max_depth**, 4),
    - (**max_leaf_nodes**, 2),
    - (**min_impurity_decrease**, 0.0),
    - (**min_samples_leaf**, 0.05),
    - (**min_samples_split**, 0.13068142884371958),
    - (**min_weight_fraction_leaf**, 0.0)])

- ## Gradient Boosting Classifier

  - **loss** -> exponential,
    
    **criterion** -> mse,
   
    **best score** = 0.8191011235955056,
    
    **melhores params**:
    - (**learning_rate**, 0.007045471947588707),
    - (**max_depth**, 20),
    - (**min_samples_leaf**, 0.05),
    - (**min_samples_split**, 0.25)

- ## Extra trees Classifier

  - **class_weight** -> balanced,
    
    **criterion** -> gini,
    
    **best score** = 0.7887640449438202,
    
    **melhores params**:
    - (**max_depth**, 4),
    - (**max_leaf_nodes**, 2),
    - (**min_impurity_decrease**, 0.0),
    - (**min_samples_leaf**, 0.05),
    - (**min_samples_split**, 0.13068142884371958),
    - (**min_weight_fraction_leaf**, 0.0)])

- ## LGBM Classifier

  - **best score** = 0.8258426966292135,
    
    **melhores params**:
    - (**colsample_bytree**, 1.0),
    - (**learning_rate**, 0.06658562101196022),
    - (**min_child_samples**, 68),
    - (**num_leaves**, 128),
    - (**subsample**, 0.05)

- ## XGB Classifier

  - **best score** = 0.8314606741573034,
    
    **melhores params**:
    - (**colsample_bylevel**, 1.0),
    - (**colsample_bytree**, 0.8148220656676483),
    - (**learning_rate**, 0.043342234186430975),
    - (**max_depth**, 20),
    - (**min_child_weight**, 4),
    - (**reg_alpha**, 1.0),
    - (**reg_lambda**, 0.46614109945918136),
    - (**subsample**, 1.0)



### Seleção de features

In [ ]:
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression, f_classif

In [ ]:
# melhores modelos até agora
modelo2 = GradientBoostingClassifier(loss='exponential', criterion='mse',
                                     learning_rate=0.007045471947588707,
                                     max_depth=20, min_samples_leaf=0.05,
                                     min_samples_split=0.25, random_state=0,
                                     n_estimators=300)

modelo9 = LGBMClassifier(colsample_bytree=1.0, learning_rate=0.06658562101196022,
                         min_child_samples=68, num_leaves=128, subsample=0.05,
                         random_state=0, n_estimators=300)

modelo10 = XGBClassifier(colsample_bytree=0.8148220656676483, colsample_bylevel=1.0,
                         learning_rate=0.043342234186430975, max_depth=20,
                         min_child_weight=4, reg_alpha=1.0,
                         reg_lambda=0.46614109945918136, subsample=1.0, 
                         random_state=0, n_estimators=300)

In [ ]:
var_menor_erro = None
valor_menor_erro = 1000
valor_menor_erro_geral = 1000
best_features =[]
lista = X_train.columns.to_list()

for ind in range(len(X_train.columns)):
  print(f'Escolhendo a variável {ind+1}')
  for var in lista:
    #mdl = RandomForestRegressor(n_estimators=500, random_state=0, n_jobs=-1)
    modelo10.fit(X_train[best_features + [var]], y_train)
    p = modelo10.predict(X_val[best_features + [var]])

    erro = accuracy_score(y_val, p)

    #print(f'Variável: {var} - Erro: {erro:.4f}\n')

    if -erro < valor_menor_erro:
      var_menor_erro = var
      valor_menor_erro = -erro
  
  print(f'Melhor Variável: {var_menor_erro} - Erro: {valor_menor_erro:.5f}\n')  
  
   
  if valor_menor_erro < valor_menor_erro_geral:
    valor_menor_erro_geral = valor_menor_erro
    best_features.append(var_menor_erro)
    lista.remove(var_menor_erro)
  else:
    print('Nenhuma variável melhora o score\n')
    break

print(f'Melhore(s) Variável(is): {best_features}')

Escolhendo a variável 1
Melhor Variável: Sex_code - Erro: -0.77578

Escolhendo a variável 2
Melhor Variável: Family_size  - Erro: -0.78924

Escolhendo a variável 3
Melhor Variável: Age_band_code  - Erro: -0.80269

Escolhendo a variável 4
Melhor Variável: Fare  - Erro: -0.82511

Escolhendo a variável 5
Melhor Variável: Sex_male - Erro: -0.82960

Escolhendo a variável 6
Melhor Variável: Age_band_code_3.0 - Erro: -0.83857

Escolhendo a variável 7
Melhor Variável: Age_band_code_3.0 - Erro: -0.83857

Nenhuma variável melhora o score

Melhore(s) Variável(is): ['Sex_code', 'Family_size ', 'Age_band_code ', 'Fare ', 'Sex_male', 'Age_band_code_3.0']


Gradient Boosting Classifier: score = 0.80269

Melhores Variáveis: ['Sex_code', 'Age_band_code*Class ', 'Name_title_code', 'Name_title_Mr', 'Fare_per_person_band_code_5.0']

LGBM Classifier: score = 0.82063

Melhores Variáveis:['Sex_code', 'Age ', 'Fare_per_person_band_code', 'Age_band_code_3.0']

XGB Classifier: score = 0.83857

Melhores variáveis: ['Sex_code', 'Family_size ', 'Age_band_code ', 'Fare ', 'Sex_male', 'Age_band_code_3.0']


In [ ]:
variaveis = ['Sex_code', 'Family_size ', 'Age_band_code ', 'Fare ', 'Sex_male', 'Age_band_code_3.0']
modelo10.fit(X_train[variaveis], y_train)
pred = modelo10.predict(X_val[variaveis])
score = accuracy_score(y_val, pred)
score

0.8385650224215246

In [ ]:
pred_final = modelo10.predict(Xtest_gg[variaveis])
pred_final

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [ ]:
# Criando submissão
sub = pd.Series(pred_final, index=test['PassengerId'], name='Survived') # modelo 2 só tunado
sub.to_csv('gender_submission_provided28.csv', header=True)

# Score na competição: 0.77511

## Testes despretenciosos

In [ ]:
def select_features(modelo):
  k_vs_score = []
  for k in range(1,len(Xtrain_gg.columns)):
    selector = SelectKBest(score_func=f_classif, k=k)

    X_train2 = selector.fit_transform(X_train, y_train)
    X_val2 = selector.transform(X_val)

    modelo.fit(X_train2, y_train)
    
    pred_sf = modelo.predict(X_val2)
    
    score_sf = accuracy_score(y_val, pred_sf)
    k_vs_score.append(-score_sf)
    print(f'k = {k} - Accuracy = {score_sf}')

  return k_vs_score


In [ ]:
plot2 = select_features(modelo2)
plot9 = select_features(modelo9)
plot10 = select_features(modelo10)


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(plot2)
plt.plot(plot9)
plt.plot(plot10)
plt.title('Feature selection', size=20)
plt.xlabel('Qte. features', size=12)
plt.ylabel('Neg. accuracy', size=12)
plt.legend()
plt.show()

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

k_vs_score3 = []
#for k in range(2, len(Xtrain_gg.columns)):
  #selector_model = LinearRegression(normalize=True) # muita feature
  #selector_model = Ridge(alpha=1.0, normalize=True) #não foi melhor que o linear-reg
selector_model = Lasso(alpha=1.0, normalize=True) # Foi o melhor pra esse caso ( e aparentemente para vários outros)
  #selector_model = XGBClassifier(random_state=0, n_jobs=-1) # As vezes o que escolhe é o que treina... as vezes
k = 31
selector = SelectFromModel(selector_model, max_features=k, threshold=-np.inf)

selector.fit(X_train, y_train)
X_train_cont = selector.transform(X_train)
X_val_cont = selector.transform(X_val)

modelo2.fit(X_train_cont, y_train)

predicoes = modelo2.predict(X_val_cont)

score = accuracy_score(y_val, predicoes)

print(f'k = {k} - Accuracy = {score}')
k_vs_score3.append(score)


In [ ]:
# modelo 2: lasso, 31 < k < 68
# modelo 9: lasso 13 < k < 30 e k = 69, 70
#modelo 10: lasso 14 < k < 16 e 40 < k < 54

plt.plot(k_vs_score1)
plt.plot(k_vs_score2)
plt.plot(k_vs_score3)
plt.plot(k_vs_score4)

In [ ]:
mask = selector.get_support()
columns_model_2 = X_train.columns[mask]


In [ ]:
mask = selector.get_support()
columns_model_9 = X_train.columns[mask]


In [ ]:
mask = selector.get_support()
columns_model_10 = X_train.columns[mask]


In [ ]:
len(columns_model_2), len(columns_model_9), len(columns_model_10)

In [ ]:
# teste doido para o modelo 2... só pra ganhar xp

fs = []
for seed in range(1000):
  np.random.seed(seed)
  k = np.random.randint(1, 71, 1)[0]
  selected = np.random.choice(X_train.columns, k, replace=False)
  #print(selected)
  
  X_train2 = X_train[selected]
  X_val2 = X_val[selected]
  
  modelo10.fit(X_train2, y_train)

  p = modelo10.predict(X_val2)

  score = accuracy_score(y_val, p)
  if score > 0.81:
    print(f"seed = {seed} - k = {k} - MAE = {score}")
    fs.append(p)

In [ ]:
np.random.seed(254)
k = 42
columns_model_10_2 = np.random.choice(X_train.columns, k, replace=False)
columns_model_10_2

In [ ]:
modelo2.fit(Xtrain_gg[columns_model_2], ytrain_gg)
pred2 = modelo2.predict(Xtest_gg[columns_model_2])

In [ ]:
modelo9.fit(Xtrain_gg[columns_model_9], ytrain_gg)
pred9 = modelo9.predict(Xtest_gg[columns_model_9])

In [ ]:
modelo10.fit(Xtrain_gg[columns_model_10], ytrain_gg)
pred10 = modelo10.predict(Xtest_gg[columns_model_10])

In [ ]:
np.random.seed(254)
modelo10.fit(Xtrain_gg[columns_model_10_2], ytrain_gg)
pred10_2 = modelo10.predict(Xtest_gg[columns_model_10_2])